# Blechpy Poisson HMM Tutorial
This tutorial will cover how to setup and fit spike data to a poisson HMM.
To use this you must first already have a blechpy.dataset object created with data that is past the spike sorting stage. 